In [1]:
import pmdarima as pm
import matplotlib.pyplot as pyplot
import pyramid

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from  datetime import datetime
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')


In [3]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',150)

In [4]:
train_data = pd.read_excel(r"../data/A_data.xlsx")
test_data = pd.read_excel(r"../data/submit.xlsx")

In [5]:
train_data["小区编号"].unique()

array([26019014, 26019015, 26019016, 26019023, 26019024, 26019010,
       26019011, 26019012, 26019013, 26019057, 26019058, 26019009,
       26019007, 26019008, 26019030, 26019031, 26019032, 26019004,
       26019005, 26019006, 26019021, 26019022, 26019020, 26019025,
       26019017, 26019018, 26019019, 26019043, 26019044, 26019040,
       26019041, 26019042, 26019001, 26019002, 26019003, 26019037,
       26019038, 26019039, 26019046, 26019047, 26019048, 26019054,
       26019055, 26019056, 26019051, 26019052, 26019053, 26019036,
       26019045, 26019049, 26019050, 26019029, 26019026, 26019033,
       26019034, 26019035, 26019027, 26019028], dtype=int64)

In [6]:
used_cols = ["时间","基站编号","小区编号","小区内的平均用户数","小区PDCP层所发送的下行数据的总吞吐量比特",
"小区PDCP层所接收到的上行数据的总吞吐量比特","平均激活用户数"]

In [7]:
train = train_data[used_cols]

In [8]:
import fancyimpute
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, IterativeImputer, BiScaler

In [11]:
now_target_col = "小区内的平均用户数"

In [12]:
X_filled_knn = KNN(k=3).fit_transform(train_data["小区内的平均用户数"].values.reshape(1,-1))

Imputing row 1/1 with 0 missing, elapsed time: 0.017


D:\Anaconda\envs\MDandNLP\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


In [13]:
X_filled_knn.shape

(1, 40368)

In [14]:
mean_user_anomaly = np.loadtxt(r"../data/target1_label.txt")

In [15]:
community_id = train["小区编号"].unique()

In [16]:
def change_anomaly_point(train_df, target_col,ad_label):
    out_arr = np.zeros((58, 696))
    for i in range(len(community_id)):
        data_series = train[train["小区编号"] == community_id[i]][target_col].values
        X_filled_knn = KNN(k=3).fit_transform(data_series.reshape(1,-1))[0]
        data_series[7:][[ad_label[i] == 1]] = X_filled_knn[7:][[ad_label[i] == 1]]
        out_arr[i] = data_series
    return out_arr

In [17]:
test_data.columns

Index(['时间', '基站编号', '小区编号', '本地小区标识', '小区内的平均用户数', '小区PDCP层所发送的下行数据的总吞吐量比特',
       '小区PDCP层所接收到的上行数据的总吞吐量比特', '平均激活用户数'],
      dtype='object')

In [18]:
mean_user_pre_time = change_anomaly_point(train_data, now_target_col,mean_user_anomaly)

Imputing row 1/1 with 0 missing, elapsed time: 0.001
Imputing row 1/1 with 0 missing, elapsed time: 0.000
Imputing row 1/1 with 0 missing, elapsed time: 0.000
Imputing row 1/1 with 0 missing, elapsed time: 0.000
Imputing row 1/1 with 0 missing, elapsed time: 0.001
Imputing row 1/1 with 0 missing, elapsed time: 0.001
Imputing row 1/1 with 0 missing, elapsed time: 0.001
Imputing row 1/1 with 0 missing, elapsed time: 0.001
Imputing row 1/1 with 0 missing, elapsed time: 0.001
Imputing row 1/1 with 0 missing, elapsed time: 0.001
Imputing row 1/1 with 0 missing, elapsed time: 0.000
Imputing row 1/1 with 0 missing, elapsed time: 0.000
Imputing row 1/1 with 0 missing, elapsed time: 0.001
Imputing row 1/1 with 0 missing, elapsed time: 0.001
Imputing row 1/1 with 0 missing, elapsed time: 0.001
Imputing row 1/1 with 0 missing, elapsed time: 0.000
Imputing row 1/1 with 0 missing, elapsed time: 0.001
Imputing row 1/1 with 0 missing, elapsed time: 0.000
Imputing row 1/1 with 0 missing, elapsed time:

D:\Anaconda\envs\MDandNLP\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\Daran\AppData\Local\Temp/ipykernel_24548/3817280434.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data_series[7:][[ad_label[i] == 1]] = X_filled_knn[7:][[ad_label[i] == 1]]
C:\Users\Daran\AppData\Local\Temp/ipykernel_24548/3817280434.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data_series[7:][[ad_label[i] == 1]] = X_filled_knn[7:][[ad_labe

In [ ]:
mean_user_pre_time

In [19]:
mean_user_pre_time.shape

(58, 696)

In [20]:
def mape(y_true, y_pred):
    """
    参数:
    y_true -- 测试集目标真实值
    y_pred -- 测试集目标预测值
    
    返回:
    mape -- MAPE 评价指标
    """
    
    n = len(y_true)
    mape = np.mean(sum(np.abs((y_true - y_pred)/y_true))/n*100)
    return mape

模拟train和val的过程，实质在验证auto_arima的拟合效果
由于本地电脑机能有限，实际生成结果的代码在远程电脑运行，因此此代码可以运行并生成结果但是并没有运行状态保留

In [21]:
all_mape = []
all_forecast = []
all_val = []
for i in range(len(community_id)):
    all_data = mean_user_pre_time[i]
    train_data = all_data[:len(all_data)-72]
    val_data = all_data[len(all_data)-72:]

    model = pm.auto_arima(train_data, seasonal=True, m=24)
    forecasts = model.predict(72)

    print(mape(val_data, forecasts))
    loss = mape(val_data, forecasts)

    all_mape.append(loss)
    all_forecast.append(forecasts)
    all_val.append(val_data)

15.36777312730857
20.267291951655007
10.569683879271622
1406.1911394134934
2061.1742224221002
35.6939843526144
15.583462414837987
20.23830447395234


C:\Users\Daran\AppData\Local\Temp/ipykernel_24548/1423323665.py:12: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(sum(np.abs((y_true - y_pred)/y_true))/n*100)
C:\Users\Daran\AppData\Local\Temp/ipykernel_24548/1423323665.py:12: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(sum(np.abs((y_true - y_pred)/y_true))/n*100)


inf
67.75100252870236
36.12879359793623


C:\Users\Daran\AppData\Local\Temp/ipykernel_24548/1423323665.py:12: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(sum(np.abs((y_true - y_pred)/y_true))/n*100)
C:\Users\Daran\AppData\Local\Temp/ipykernel_24548/1423323665.py:12: RuntimeWarning: divide by zero encountered in true_divide
  mape = np.mean(sum(np.abs((y_true - y_pred)/y_true))/n*100)


inf
13.516387846222408
14.662504578062624
16.11899144733937
25.024648651167613
19.31151863390862
33.6959796085601
21.650924671119185
37.17341763002035
20.73150370848217
9.989969951116437


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
all_mape

In [ ]:
all_forecast

进行真实预测并且将结果写入提交文件中

In [ ]:
all_forecast = []
all_val = []
for i in range(len(community_id)):
    all_data = mean_user_pre_time[i]
    train_data = all_data[:]
   
    model = pm.auto_arima(train_data, seasonal=True, m=24)
    forecasts = model.predict(72)
    
    test_data[test_data["小区编号"] == community_id[i]][now_target_col] = forecasts

    all_forecast.append(forecasts)
    
    

In [ ]:
test_data